In [1]:
# -*- coding: utf-8 -*-

import argparse
import shutil

import numpy as np
import torch
from torch.autograd import Variable, grad
from torch.nn.init import xavier_normal
from torchvision import datasets, transforms
import torchvision.utils as vutils

from data import *
from models import *
from util import *
import time

In [2]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def adjust_learning_rate(optimizer, epoch, g):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    if g == 'g':
        lr = learning_rate * (0.5 ** (epoch // 10))
    else:
        lr = learning_rate * ((0.5*0.9) ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


Load dataset

In [3]:
print('Preparing dataset A...')
A_train_dataset = AttrDataset('./extra_data/reduced_hair.csv', DATA_PATH, transform=face_transform)
train_loader = DataLoader(A_train_dataset, batch_size=batch_size, num_workers=8, shuffle=True, drop_last=True)

Preparing dataset A...


Define the network and optimizer.

In [4]:
generator = Generator()
generator.apply(weights_init)

discriminator = Discriminator2()
discriminator.apply(weights_init)

opt_g = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
opt_d = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

Start from checkpoint

In [5]:
#resume_file = './models_big/Epoch:_049.pt'
resume_file = './models/Epoch:_008.pt'
if resume_file:
    if os.path.isfile(resume_file):
        print("=> loading checkpoint '{}'".format(resume_file))
        checkpoint = torch.load(resume_file)
        start_epoch = checkpoint['epoch'] + 1
        generator.load_state_dict(checkpoint['g_state_dict'])
        discriminator.load_state_dict(checkpoint['d_state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
               .format(resume_file, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resume_file))

=> loading checkpoint './models/Epoch:_008.pt'
=> loaded checkpoint './models/Epoch:_008.pt' (epoch 8)


In [6]:
start_epoch

9

Define loss and input of network.<br>
Move variables onto GPU.

In [5]:
criterion = torch.nn.BCEWithLogitsLoss() # StableBCELoss() #
X = Variable(torch.FloatTensor(batch_size, 3, imsize, imsize))
z = Variable(torch.FloatTensor(batch_size, z_dim))
tags = Variable(torch.FloatTensor(batch_size, tag_num))
labels = Variable(torch.FloatTensor(batch_size))
tag = torch.LongTensor(batch_size)
if cuda:
    generator.cuda()
    discriminator.cuda()
    criterion.cuda()
    X, z, tags, labels,tag = X.cuda(), z.cuda(), tags.cuda(), labels.cuda(),tag.cuda()
    

In [6]:
from torchsummary import summary
summary(generator,(1,128+4))
#summary(discriminator,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 1, 16384]       2,179,072
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]          36,928
       BatchNorm2d-5           [-1, 64, 16, 16]             128
              ReLU-6           [-1, 64, 16, 16]               0
            Conv2d-7           [-1, 64, 16, 16]          36,928
       BatchNorm2d-8           [-1, 64, 16, 16]             128
     residualBlock-9           [-1, 64, 16, 16]               0
           Conv2d-10           [-1, 64, 16, 16]          36,928
      BatchNorm2d-11           [-1, 64, 16, 16]             128
             ReLU-12           [-1, 64, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          36,928
      BatchNorm2d-14           [-1, 64,

make embedding input vector

In [9]:
allone = [1.0 for i in range(32)]
allzero =[0.0 for i in range(32)]
def edit(tar,allone,allzero):
    out = []
    for i in tar:
        tmp = []
        for j in range(4):
            if j == i:
                tmp += allzero
            else:
                tmp += allone
        out.append(tmp)
    out = torch.FloatTensor(out)
    return out.cuda()

In [10]:
# Train
best_loss = float('inf')
#embedding = nn.Embedding(batch_size,128).cuda()
k = 1
max_epochs += start_epoch
for epoch in range(start_epoch, max_epochs):

    adjust_learning_rate(opt_g, epoch,'g')
    adjust_learning_rate(opt_d, epoch,'d')

    for batch_idx, (data, target) in enumerate(train_loader):
        if k%5 == 0:
            k = 0
        if epoch > 20:
            k += 1
        
        X.data.copy_(data)

        # Update discriminator
        # train with real
        tags.data.copy_(target)
        discriminator.zero_grad()
        pred_real, pred_real_tag = discriminator(X)
        labels.data.fill_(1.0)
        loss_d_real_label = criterion(torch.squeeze(pred_real), labels)
        loss_d_real_tag = criterion(pred_real_tag, tags)
        loss_d_real = lambda_adv * loss_d_real_label + loss_d_real_tag
        #if (k-1)%5 == 0:
        loss_d_real.backward()

        # train with fake
        z.data.normal_(0, 1)
        np_tar = np.argmax(target.numpy(),axis = 1)
        tag = torch.LongTensor(np_tar).cuda()
        #embed = embedding(tag).view(batch_size,-1)
        #xx = z.mul(embed)
        
        reduced = edit(np_tar,allone,allzero)
        x = z.mul(reduced)
        
        rep = torch.cat((x, tags.clone()), 1)
        fake = generator.forward(rep).detach()
        pred_fake, pred_fake_tag = discriminator(fake)
        labels.data.fill_(0.0)
        loss_d_fake_label = criterion(torch.squeeze(pred_fake), labels)
        loss_d_fake_tag = criterion(pred_fake_tag, tags)
        loss_d_fake = lambda_adv * loss_d_fake_label + loss_d_fake_tag


        loss_d_fake.backward()

        # gradient penalty
        shape = [batch_size] + [1]*(X.dim()-1)
        alpha = torch.rand(*shape)
        beta = torch.rand(X.size())
        if cuda:
            alpha = alpha.cuda()
            beta = beta.cuda()
        x_hat = Variable(alpha * X.data + (1 - alpha) * (X.data + 0.5 * X.data.std() * beta), requires_grad=True)
        pred_hat, _ = discriminator(x_hat)
        grad_out = torch.ones(pred_hat.size())
        if cuda:
            grad_out = grad_out.cuda()
        gradients = grad(outputs=pred_hat, inputs=x_hat, grad_outputs=grad_out,
                create_graph=True, retain_graph=True, only_inputs=True)[0]
        gradient_penalty = lambda_gp * ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        #if (k-1)%5 == 0:
        gradient_penalty.backward()

        loss_d = loss_d_real + loss_d_fake + gradient_penalty
        opt_d.step()

        # Update generator
        generator.zero_grad()
        z.data.normal_(0, 1)
        np_tar = np.argmax(target.numpy(),axis = 1)
        tag = torch.LongTensor(np_tar).cuda()
        #embed = embedding(tag).view(batch_size,-1)
        #xx = z.mul(embed)
        
        reduced = edit(np_tar,allone,allzero)
        x = z.mul(reduced)
        
        
        #tags.data.uniform_(to=1) # Continuous
        # tags.data.fill_(0.5)           # Discrete binary string
        # tags.data.bernoulli_()
        rep = torch.cat((x, tags.clone()), 1)
        gen = generator(rep)
        pred_gen, pred_gen_tag = discriminator(gen)
        labels.data.fill_(1)

        loss_g_label = criterion(torch.squeeze(pred_gen), labels)

        loss_g_tag = criterion(pred_gen_tag, tags)
        loss_g = lambda_adv * loss_g_label + loss_g_tag
        loss_g.backward()
        opt_g.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f Loss_D_Label: %.4f Loss_G_Label: %.4f Loss_D_Tag: %.4f Loss_G_Tag: %.4f'
              % (epoch, max_epochs, batch_idx, len(train_loader),
                 loss_d.item(), loss_g.item(), loss_d_real_label.item()+loss_d_fake_label.item(),
                 loss_g_label.item(), loss_d_real_tag.item()+loss_d_fake_tag.item(),
                 loss_g_tag.item()))

        if batch_idx % 100 == 0:
            vutils.save_image(data,
                    'samples/real_samples_epoch_%03d.png' % epoch)
            fake = generator(rep)
            vutils.save_image(fake.data.view(batch_size, 3, imsize, imsize),
                    'samples/fake_samples_epoch_%03d.png' % epoch)

            is_best = False

            save_checkpoint({
                'epoch': epoch,
                'arch': 'ACGAN-SRResNet-DRAGAN',
                'g_state_dict': generator.state_dict(),
                'd_state_dict': discriminator.state_dict(),
                'loss_g': loss_g.item(),
                'loss_d': loss_d.item(),
                'g_optimizer' : opt_g.state_dict(),
                'd_optimizer' : opt_d.state_dict(),
            }, is_best, 'models/Epoch:_%03d.pt' % (epoch))

[9/59][0/243] Loss_D: 3.9356 Loss_G: 4.9250 Loss_D_Label: 0.8474 Loss_G_Label: 1.2301 Loss_D_Tag: 0.0515 Loss_G_Tag: 0.0048
[9/59][1/243] Loss_D: 4.8198 Loss_G: 6.3648 Loss_D_Label: 1.0602 Loss_G_Label: 1.5875 Loss_D_Tag: 0.0862 Loss_G_Tag: 0.0146
[9/59][2/243] Loss_D: 4.7193 Loss_G: 5.7036 Loss_D_Label: 1.0478 Loss_G_Label: 1.4250 Loss_D_Tag: 0.0348 Loss_G_Tag: 0.0037
[9/59][3/243] Loss_D: 4.4298 Loss_G: 5.0008 Loss_D_Label: 0.9661 Loss_G_Label: 1.2499 Loss_D_Tag: 0.0734 Loss_G_Tag: 0.0010
[9/59][4/243] Loss_D: 4.9882 Loss_G: 5.1685 Loss_D_Label: 1.1000 Loss_G_Label: 1.2912 Loss_D_Tag: 0.0974 Loss_G_Tag: 0.0038
[9/59][5/243] Loss_D: 4.6049 Loss_G: 6.9221 Loss_D_Label: 1.0051 Loss_G_Label: 1.7194 Loss_D_Tag: 0.0936 Loss_G_Tag: 0.0446
[9/59][6/243] Loss_D: 4.0101 Loss_G: 6.4095 Loss_D_Label: 0.8604 Loss_G_Label: 1.5998 Loss_D_Tag: 0.0776 Loss_G_Tag: 0.0102
[9/59][7/243] Loss_D: 4.9961 Loss_G: 7.2143 Loss_D_Label: 1.1137 Loss_G_Label: 1.8008 Loss_D_Tag: 0.0519 Loss_G_Tag: 0.0112
[9/59][8

[9/59][66/243] Loss_D: 4.7045 Loss_G: 9.8907 Loss_D_Label: 1.0325 Loss_G_Label: 2.4632 Loss_D_Tag: 0.0859 Loss_G_Tag: 0.0379
[9/59][67/243] Loss_D: 4.3378 Loss_G: 7.6164 Loss_D_Label: 0.9271 Loss_G_Label: 1.8839 Loss_D_Tag: 0.1401 Loss_G_Tag: 0.0807
[9/59][68/243] Loss_D: 3.6977 Loss_G: 5.9146 Loss_D_Label: 0.7725 Loss_G_Label: 1.4768 Loss_D_Tag: 0.1190 Loss_G_Tag: 0.0074
[9/59][69/243] Loss_D: 5.0950 Loss_G: 7.6984 Loss_D_Label: 1.1288 Loss_G_Label: 1.9157 Loss_D_Tag: 0.0915 Loss_G_Tag: 0.0355
[9/59][70/243] Loss_D: 4.3492 Loss_G: 7.3080 Loss_D_Label: 0.9266 Loss_G_Label: 1.8255 Loss_D_Tag: 0.1551 Loss_G_Tag: 0.0062
[9/59][71/243] Loss_D: 3.9596 Loss_G: 5.7501 Loss_D_Label: 0.8619 Loss_G_Label: 1.4118 Loss_D_Tag: 0.0238 Loss_G_Tag: 0.1030
[9/59][72/243] Loss_D: 3.9793 Loss_G: 7.9653 Loss_D_Label: 0.8567 Loss_G_Label: 1.9753 Loss_D_Tag: 0.0653 Loss_G_Tag: 0.0642
[9/59][73/243] Loss_D: 4.1310 Loss_G: 8.7633 Loss_D_Label: 0.9035 Loss_G_Label: 2.1863 Loss_D_Tag: 0.0305 Loss_G_Tag: 0.0180


[9/59][132/243] Loss_D: 5.4997 Loss_G: 6.2477 Loss_D_Label: 1.2125 Loss_G_Label: 1.4575 Loss_D_Tag: 0.1592 Loss_G_Tag: 0.4175
[9/59][133/243] Loss_D: 4.2172 Loss_G: 5.9001 Loss_D_Label: 0.8885 Loss_G_Label: 1.4584 Loss_D_Tag: 0.1739 Loss_G_Tag: 0.0666
[9/59][134/243] Loss_D: 4.1484 Loss_G: 8.3628 Loss_D_Label: 0.8768 Loss_G_Label: 2.0829 Loss_D_Tag: 0.1512 Loss_G_Tag: 0.0310
[9/59][135/243] Loss_D: 4.0164 Loss_G: 7.9996 Loss_D_Label: 0.7786 Loss_G_Label: 1.9736 Loss_D_Tag: 0.4106 Loss_G_Tag: 0.1052
[9/59][136/243] Loss_D: 4.3403 Loss_G: 6.4892 Loss_D_Label: 0.9195 Loss_G_Label: 1.6033 Loss_D_Tag: 0.1719 Loss_G_Tag: 0.0759
[9/59][137/243] Loss_D: 4.3854 Loss_G: 9.1602 Loss_D_Label: 0.9122 Loss_G_Label: 2.2786 Loss_D_Tag: 0.2470 Loss_G_Tag: 0.0457
[9/59][138/243] Loss_D: 3.8945 Loss_G: 6.8725 Loss_D_Label: 0.7723 Loss_G_Label: 1.7024 Loss_D_Tag: 0.3158 Loss_G_Tag: 0.0630
[9/59][139/243] Loss_D: 4.9367 Loss_G: 8.9350 Loss_D_Label: 1.0869 Loss_G_Label: 2.2278 Loss_D_Tag: 0.1014 Loss_G_Tag:

[9/59][197/243] Loss_D: 6.1121 Loss_G: 7.5858 Loss_D_Label: 1.3618 Loss_G_Label: 1.8858 Loss_D_Tag: 0.1738 Loss_G_Tag: 0.0427
[9/59][198/243] Loss_D: 4.0043 Loss_G: 4.6731 Loss_D_Label: 0.8469 Loss_G_Label: 1.1555 Loss_D_Tag: 0.1249 Loss_G_Tag: 0.0509
[9/59][199/243] Loss_D: 4.1619 Loss_G: 6.1318 Loss_D_Label: 0.8782 Loss_G_Label: 1.5198 Loss_D_Tag: 0.1575 Loss_G_Tag: 0.0527
[9/59][200/243] Loss_D: 4.5478 Loss_G: 7.0967 Loss_D_Label: 0.9529 Loss_G_Label: 1.7616 Loss_D_Tag: 0.2450 Loss_G_Tag: 0.0504
[9/59][201/243] Loss_D: 4.4568 Loss_G: 7.2295 Loss_D_Label: 0.9477 Loss_G_Label: 1.7979 Loss_D_Tag: 0.1754 Loss_G_Tag: 0.0378
[9/59][202/243] Loss_D: 4.2648 Loss_G: 6.1844 Loss_D_Label: 0.9025 Loss_G_Label: 1.5249 Loss_D_Tag: 0.1641 Loss_G_Tag: 0.0847
[9/59][203/243] Loss_D: 5.2946 Loss_G: 7.6006 Loss_D_Label: 1.1085 Loss_G_Label: 1.8931 Loss_D_Tag: 0.3709 Loss_G_Tag: 0.0281
[9/59][204/243] Loss_D: 4.0924 Loss_G: 7.8106 Loss_D_Label: 0.8507 Loss_G_Label: 1.9431 Loss_D_Tag: 0.1995 Loss_G_Tag:

[10/59][20/243] Loss_D: 3.0035 Loss_G: 8.1528 Loss_D_Label: 0.6146 Loss_G_Label: 2.0367 Loss_D_Tag: 0.0592 Loss_G_Tag: 0.0060
[10/59][21/243] Loss_D: 3.7729 Loss_G: 9.2435 Loss_D_Label: 0.7970 Loss_G_Label: 2.3082 Loss_D_Tag: 0.0996 Loss_G_Tag: 0.0107
[10/59][22/243] Loss_D: 2.9812 Loss_G: 9.2401 Loss_D_Label: 0.6101 Loss_G_Label: 2.3027 Loss_D_Tag: 0.0552 Loss_G_Tag: 0.0293
[10/59][23/243] Loss_D: 2.8778 Loss_G: 8.0815 Loss_D_Label: 0.5826 Loss_G_Label: 2.0157 Loss_D_Tag: 0.0609 Loss_G_Tag: 0.0189
[10/59][24/243] Loss_D: 2.4545 Loss_G: 9.9282 Loss_D_Label: 0.4780 Loss_G_Label: 2.4757 Loss_D_Tag: 0.0571 Loss_G_Tag: 0.0254
[10/59][25/243] Loss_D: 3.6118 Loss_G: 9.7845 Loss_D_Label: 0.7533 Loss_G_Label: 2.4445 Loss_D_Tag: 0.1133 Loss_G_Tag: 0.0067
[10/59][26/243] Loss_D: 2.8104 Loss_G: 9.3524 Loss_D_Label: 0.5659 Loss_G_Label: 2.3305 Loss_D_Tag: 0.0622 Loss_G_Tag: 0.0302
[10/59][27/243] Loss_D: 3.0554 Loss_G: 10.6015 Loss_D_Label: 0.6051 Loss_G_Label: 2.6396 Loss_D_Tag: 0.1499 Loss_G_Tag

...

[57/59][169/243] Loss_D: 0.7970 Loss_G: 26.7845 Loss_D_Label: 0.1024 Loss_G_Label: 6.6961 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[57/59][170/243] Loss_D: 0.8604 Loss_G: 31.3600 Loss_D_Label: 0.1195 Loss_G_Label: 7.8400 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[57/59][171/243] Loss_D: 1.7586 Loss_G: 31.4028 Loss_D_Label: 0.3439 Loss_G_Label: 7.8507 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[57/59][172/243] Loss_D: 0.7359 Loss_G: 29.3363 Loss_D_Label: 0.0872 Loss_G_Label: 7.3341 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[57/59][173/243] Loss_D: 0.9616 Loss_G: 30.1906 Loss_D_Label: 0.1444 Loss_G_Label: 7.5476 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[57/59][174/243] Loss_D: 0.8345 Loss_G: 30.5677 Loss_D_Label: 0.1105 Loss_G_Label: 7.6419 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[57/59][175/243] Loss_D: 0.8387 Loss_G: 31.5114 Loss_D_Label: 0.1128 Loss_G_Label: 7.8777 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0004
[57/59][176/243] Loss_D: 1.1111 Loss_G: 30.6586 Loss_D_Label: 0.1818 Loss_G_Label: 7.6646 Loss_D_Tag: 0.

[57/59][234/243] Loss_D: 0.7546 Loss_G: 29.0952 Loss_D_Label: 0.0914 Loss_G_Label: 7.2738 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[57/59][235/243] Loss_D: 0.6918 Loss_G: 30.5214 Loss_D_Label: 0.0763 Loss_G_Label: 7.6303 Loss_D_Tag: 0.0003 Loss_G_Tag: 0.0001
[57/59][236/243] Loss_D: 0.9121 Loss_G: 29.0616 Loss_D_Label: 0.1318 Loss_G_Label: 7.2654 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[57/59][237/243] Loss_D: 1.0272 Loss_G: 29.3273 Loss_D_Label: 0.1595 Loss_G_Label: 7.3318 Loss_D_Tag: 0.0020 Loss_G_Tag: 0.0000
[57/59][238/243] Loss_D: 0.7543 Loss_G: 29.0372 Loss_D_Label: 0.0919 Loss_G_Label: 7.2593 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[57/59][239/243] Loss_D: 1.1781 Loss_G: 30.2446 Loss_D_Label: 0.1992 Loss_G_Label: 7.5611 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[57/59][240/243] Loss_D: 0.7927 Loss_G: 32.7409 Loss_D_Label: 0.1008 Loss_G_Label: 8.1852 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[57/59][241/243] Loss_D: 0.9930 Loss_G: 32.1818 Loss_D_Label: 0.1510 Loss_G_Label: 8.0453 Loss_D_Tag: 0.

[58/59][56/243] Loss_D: 0.7501 Loss_G: 27.9416 Loss_D_Label: 0.0910 Loss_G_Label: 6.9854 Loss_D_Tag: 0.0003 Loss_G_Tag: 0.0000
[58/59][57/243] Loss_D: 1.1834 Loss_G: 29.6203 Loss_D_Label: 0.1990 Loss_G_Label: 7.4051 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][58/243] Loss_D: 1.0353 Loss_G: 29.4311 Loss_D_Label: 0.1616 Loss_G_Label: 7.3578 Loss_D_Tag: 0.0010 Loss_G_Tag: 0.0000
[58/59][59/243] Loss_D: 1.0805 Loss_G: 30.9260 Loss_D_Label: 0.1716 Loss_G_Label: 7.7294 Loss_D_Tag: 0.0052 Loss_G_Tag: 0.0083
[58/59][60/243] Loss_D: 0.8807 Loss_G: 28.0003 Loss_D_Label: 0.1240 Loss_G_Label: 7.0000 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0002
[58/59][61/243] Loss_D: 1.4400 Loss_G: 30.4123 Loss_D_Label: 0.2633 Loss_G_Label: 7.6031 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[58/59][62/243] Loss_D: 0.7806 Loss_G: 27.5772 Loss_D_Label: 0.0985 Loss_G_Label: 6.8943 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][63/243] Loss_D: 0.8962 Loss_G: 28.7001 Loss_D_Label: 0.1280 Loss_G_Label: 7.1750 Loss_D_Tag: 0.0004 Los

[58/59][121/243] Loss_D: 0.7760 Loss_G: 31.1824 Loss_D_Label: 0.0973 Loss_G_Label: 7.7956 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][122/243] Loss_D: 0.8794 Loss_G: 33.3231 Loss_D_Label: 0.1239 Loss_G_Label: 8.3308 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][123/243] Loss_D: 1.0439 Loss_G: 33.0366 Loss_D_Label: 0.1642 Loss_G_Label: 8.2592 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[58/59][124/243] Loss_D: 0.7244 Loss_G: 31.2047 Loss_D_Label: 0.0858 Loss_G_Label: 7.8012 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[58/59][125/243] Loss_D: 1.2514 Loss_G: 30.1920 Loss_D_Label: 0.2180 Loss_G_Label: 7.5480 Loss_D_Tag: 0.0006 Loss_G_Tag: 0.0000
[58/59][126/243] Loss_D: 1.3108 Loss_G: 32.3995 Loss_D_Label: 0.2294 Loss_G_Label: 8.0999 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[58/59][127/243] Loss_D: 0.6603 Loss_G: 28.9888 Loss_D_Label: 0.0682 Loss_G_Label: 7.2472 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0001
[58/59][128/243] Loss_D: 0.8754 Loss_G: 31.4240 Loss_D_Label: 0.1220 Loss_G_Label: 7.8560 Loss_D_Tag: 0.

[58/59][186/243] Loss_D: 0.7235 Loss_G: 29.5526 Loss_D_Label: 0.0839 Loss_G_Label: 7.3881 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[58/59][187/243] Loss_D: 1.0192 Loss_G: 27.6910 Loss_D_Label: 0.1584 Loss_G_Label: 6.9227 Loss_D_Tag: 0.0006 Loss_G_Tag: 0.0000
[58/59][188/243] Loss_D: 0.9105 Loss_G: 29.6971 Loss_D_Label: 0.1317 Loss_G_Label: 7.4243 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][189/243] Loss_D: 1.8927 Loss_G: 29.4887 Loss_D_Label: 0.3757 Loss_G_Label: 7.3722 Loss_D_Tag: 0.0002 Loss_G_Tag: 0.0000
[58/59][190/243] Loss_D: 0.8084 Loss_G: 30.2416 Loss_D_Label: 0.1061 Loss_G_Label: 7.5604 Loss_D_Tag: 0.0004 Loss_G_Tag: 0.0000
[58/59][191/243] Loss_D: 1.2135 Loss_G: 30.3518 Loss_D_Label: 0.2073 Loss_G_Label: 7.5880 Loss_D_Tag: 0.0001 Loss_G_Tag: 0.0000
[58/59][192/243] Loss_D: 1.0543 Loss_G: 32.8322 Loss_D_Label: 0.1671 Loss_G_Label: 8.2081 Loss_D_Tag: 0.0008 Loss_G_Tag: 0.0000
[58/59][193/243] Loss_D: 0.7625 Loss_G: 31.3801 Loss_D_Label: 0.0942 Loss_G_Label: 7.8450 Loss_D_Tag: 0.